In [392]:
import pandas as pd
import numpy
import re

from jupyter_core.migrate import regex

In [393]:
events_df = pd.read_csv('events.csv')
users_df = pd.read_csv('users.csv')

In [394]:
events_df.isnull().values.any()

np.True_

In [395]:
users_df.isnull().values.any()

np.False_

In [396]:
# First checking if there are any duplicates in the user emails
duplicate_user_emails = users_df[users_df.duplicated(subset=['email']) == True ]
duplicate_user_emails

,user_id,name,email,signup_date


In [397]:
# There are no duplicates, but if we wanted to mark them we can make a new boolean column
users_df['is_dupe'] = users_df.duplicated(subset='email', keep=False)
users_df

,user_id,name,email,signup_date,is_dupe
0,0,Danielle Johnson,danielle.johnson@gmail.com,2024-09-27,False
1,1,Jeffery Wagner,jeffery.wagner@icloud.com,2024-07-26,False
2,2,Amy Robinson,amy.robinson@outlook.com,2023-04-12,False
3,3,Linda Wolfe,linda.wolfe@gmail.com,2025-03-07,False
4,4,Matthew Davis,matthew.davis@hotmail.com,2024-06-22,False
...,...,...,...,...,...
97,97,Gregory Estes White,gregory.estes@gmail.com,2024-02-19,False
98,98,Brent Wright White,brent.wright@msn.com,2024-11-13,False
99,99,Ms. Dana Cannon,dana.cannon@outlook.com,2023-10-25,False
100,100,Rick Daniels-Brown,rick.daniels@outlook.com,2024-02-09,False


In [398]:
nulldf_events = events_df.isnull()
nulldf_events

,event_id,user_email,event_type,timestamp
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
495,False,False,False,False
496,False,False,False,False
497,False,False,False,False
498,False,False,False,False


In [399]:
nulldf_users = users_df.isnull()
nulldf_users

,user_id,name,email,signup_date,is_dupe
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
97,False,False,False,False,False
98,False,False,False,False,False
99,False,False,False,False,False
100,False,False,False,False,False


In [400]:
#There are 5 event rows with NaN in the email
nanrows_events = events_df[nulldf_events.any(axis=1)]
nanrows_events


,event_id,user_email,event_type,timestamp
98,99,NaN,purchase,2023-06-16 05:11:23.034739
280,281,NaN,purchase,2025-02-10 15:34:22.668397
357,358,NaN,password_reset,2024-01-08 01:21:25.623984
396,397,NaN,password_reset,2024-08-22 04:55:50.082282
420,421,NaN,login,2023-11-10 19:33:28.225804


In [401]:
# There are no events with an empty string for user email
nanrows_events = events_df[events_df['user_email']=='']
nanrows_events

,event_id,user_email,event_type,timestamp


In [402]:
#There are no users with Nan in the email
nanrows_users = users_df[nulldf_users.any(axis=1)]
nanrows_users

,user_id,name,email,signup_date,is_dupe


In [403]:
# There are no users with an empty string for an email
nanrows_users = users_df[users_df['email']=='']
nanrows_users

,user_id,name,email,signup_date,is_dupe


In [404]:
# Joining without normalization
# We can be sure every user entry has an email based on the above investigation
# We can use right join to ensure each event is included in the output
# Events that dont join will show up with Null values for the user columns
joined_df = users_df.merge(events_df, how="right", left_on='email', right_on='user_email')
joined_df['user_id'] = joined_df['user_id'].astype('Int64')  # capital "I" for pandas nullable integer

joined_df

,user_id,name,email,signup_date,is_dupe,event_id,user_email,event_type,timestamp
0,14,Daniel Burton,daniel.burton@protonmail.com,2023-05-24,False,1,daniel.burton@protonmail.com,logout,2023-09-10 20:32:58.421093
1,<NA>,NaN,NaN,NaN,NaN,2,jenniferevans@msncom,purchase,2023-12-03 23:55:36.554694
2,29,Jessica Nunez,jessica.nunez@gmail.com,2024-01-27,False,3,jessica.nunez@gmail.com,logout,2024-10-16 07:20:12.854738
3,32,Tanya House,tanya.house@protonmail.com,2024-03-28,False,4,tanya.house@protonmail.com,password_reset,2024-01-11 04:28:54.790473
4,95,Lisa Evans-Lee,lisa.evans@msn.com,2023-07-15,False,5,lisa.evans@msn.com,login,2024-10-09 13:41:53.942650
...,...,...,...,...,...,...,...,...,...
495,67,Justin Jordan,justin.jordan@outlook.com,2024-03-12,False,496,justin.jordan@outlook.com,updated_profile,2024-01-25 01:48:58.194437
496,79,Devon Hall,devon.hall@comcast.net,2023-05-24,False,497,devon.hall@comcast.net,logout,2023-08-25 21:27:20.373314
497,32,Tanya House,tanya.house@protonmail.com,2024-03-28,False,498,tanya.house@protonmail.com,viewed_product,2023-07-07 11:28:07.010142
498,<NA>,NaN,NaN,NaN,NaN,499,barbarascott@outlookcom,login,2024-12-18 17:08:01.895173


In [405]:
# We can take a look at the rows with null emails in the user column as well as null email in the events columns
# (Recall that we found 5 events that had NaN in the email column)

nanrows_joined = joined_df[ joined_df[['email', 'user_email']].isnull().any(axis=1) ].sort_values(by=['user_email'])
nanrows_joined

,user_id,name,email,signup_date,is_dupe,event_id,user_email,event_type,timestamp
158,<NA>,NaN,NaN,NaN,NaN,159,amy.robinson@outlook.com,deleted_account,2024-04-24 23:18:50.063910
262,<NA>,NaN,NaN,NaN,NaN,263,andrea.calderon@outlook.com,logout,2023-08-02 22:25:58.806733
105,<NA>,NaN,NaN,NaN,NaN,106,barbara.scott@outlook.com,updated_profile,2024-02-14 01:33:05.969064
20,<NA>,NaN,NaN,NaN,NaN,21,chelsea.jackson@live.com,password_reset,2023-07-10 10:40:06.963536
193,<NA>,NaN,NaN,NaN,NaN,194,chelsea.jackson@live.com,password_reset,2023-05-31 09:44:45.738924
...,...,...,...,...,...,...,...,...,...
98,<NA>,NaN,NaN,NaN,NaN,99,NaN,purchase,2023-06-16 05:11:23.034739
280,<NA>,NaN,NaN,NaN,NaN,281,NaN,purchase,2025-02-10 15:34:22.668397
357,<NA>,NaN,NaN,NaN,NaN,358,NaN,password_reset,2024-01-08 01:21:25.623984
396,<NA>,NaN,NaN,NaN,NaN,397,NaN,password_reset,2024-08-22 04:55:50.082282


In [406]:
# The event_ids that did not join (without normalization)
# Ideally this number will go down post-normalization
nanrows_joined['event_id']

158    159
262    263
105    106
20      21
193    194
      ... 
98      99
280    281
357    358
396    397
420    421
Name: event_id, Length: 112, dtype: int64

In [407]:
# Out of curiosity I wanted to see if there might be a pattern in why the emails werent matched
# I used an email that I saw in the joined data set above that had an event but no user, Chelsea Jackson
# It looks like there are two password resets that didnt match, but the profile update and deleted account did
emailcheck_joined = joined_df[ joined_df['user_email'].str.contains('chelsea.jackson', na=False)  ].sort_values(by='timestamp')
emailcheck_joined

,user_id,name,email,signup_date,is_dupe,event_id,user_email,event_type,timestamp
193,<NA>,NaN,NaN,NaN,NaN,194,chelsea.jackson@live.com,password_reset,2023-05-31 09:44:45.738924
20,<NA>,NaN,NaN,NaN,NaN,21,chelsea.jackson@live.com,password_reset,2023-07-10 10:40:06.963536
429,8,Chelsea Jackson,chelsea.jackson@live.com,2024-01-10,False,430,chelsea.jackson@live.com,updated_profile,2023-10-13 22:53:57.236921
143,8,Chelsea Jackson,chelsea.jackson@live.com,2024-01-10,False,144,chelsea.jackson@live.com,deleted_account,2024-03-23 05:08:30.684076


In [408]:
# The emails were identical, so I immediately suspected some kind of invisible formatting might have been causing the issue
# It looks like at least for some emails there are white spaces, and those are causing issues
# I will use repr to print out all invisible text formatting
emailcheck_joined['repr_print'] = emailcheck_joined['user_email'].map(lambda x: repr(x))
emailcheck_joined

,user_id,name,email,signup_date,is_dupe,event_id,user_email,event_type,timestamp,repr_print
193,<NA>,NaN,NaN,NaN,NaN,194,chelsea.jackson@live.com,password_reset,2023-05-31 09:44:45.738924,' chelsea.jackson@live.com '
20,<NA>,NaN,NaN,NaN,NaN,21,chelsea.jackson@live.com,password_reset,2023-07-10 10:40:06.963536,' chelsea.jackson@live.com '
429,8,Chelsea Jackson,chelsea.jackson@live.com,2024-01-10,False,430,chelsea.jackson@live.com,updated_profile,2023-10-13 22:53:57.236921,'chelsea.jackson@live.com'
143,8,Chelsea Jackson,chelsea.jackson@live.com,2024-01-10,False,144,chelsea.jackson@live.com,deleted_account,2024-03-23 05:08:30.684076,'chelsea.jackson@live.com'


In [409]:
# Making further use of repr(), Ive found that for many of the unmatched emails there are either trailing whitespaces or unexpected characters like
# 'elizabeth\u200b.travis@hotmail\u200b.com' or 'janet.williams\xa0@\xa0icloud.com'
# there are a few emails that don't seem to have any special characters or whitespaces- those are confirmed true negatives
nanrows_joined['repr_print'] = nanrows_joined['user_email'].map(lambda x: repr(x))
nanrows_joined.sort_values(by='user_email')

,user_id,name,email,signup_date,is_dupe,event_id,user_email,event_type,timestamp,repr_print
158,<NA>,NaN,NaN,NaN,NaN,159,amy.robinson@outlook.com,deleted_account,2024-04-24 23:18:50.063910,' amy.robinson@outlook.com '
262,<NA>,NaN,NaN,NaN,NaN,263,andrea.calderon@outlook.com,logout,2023-08-02 22:25:58.806733,' andrea.calderon@outlook.com '
105,<NA>,NaN,NaN,NaN,NaN,106,barbara.scott@outlook.com,updated_profile,2024-02-14 01:33:05.969064,' barbara.scott@outlook.com '
20,<NA>,NaN,NaN,NaN,NaN,21,chelsea.jackson@live.com,password_reset,2023-07-10 10:40:06.963536,' chelsea.jackson@live.com '
193,<NA>,NaN,NaN,NaN,NaN,194,chelsea.jackson@live.com,password_reset,2023-05-31 09:44:45.738924,' chelsea.jackson@live.com '
...,...,...,...,...,...,...,...,...,...,...
98,<NA>,NaN,NaN,NaN,NaN,99,NaN,purchase,2023-06-16 05:11:23.034739,nan
280,<NA>,NaN,NaN,NaN,NaN,281,NaN,purchase,2025-02-10 15:34:22.668397,nan
357,<NA>,NaN,NaN,NaN,NaN,358,NaN,password_reset,2024-01-08 01:21:25.623984,nan
396,<NA>,NaN,NaN,NaN,NaN,397,NaN,password_reset,2024-08-22 04:55:50.082282,nan


In [410]:
# Now we can normalize the emails and see if that improves the join rate
# To normalize I will lowercase all the emails in both tables
# I will also remove any special characters/formatting/trailing spaces
# I am using the regex [^\x00-\x7F]: the range of ASCII characters is defined by \x00-\x7F and the caret ^ negates this set
# re.sub will remove anything identified above

nanrows_joined['cleaned_email'] = nanrows_joined['user_email'].astype(str).map(lambda x: re.sub(r'[^\x00-\x7F]+', '', x)).map(lambda x: x.strip()).map(lambda x: x.lower())
nanrows_joined['encoded_email_repr'] = nanrows_joined['cleaned_email'].map(lambda x: repr(x))
nanrows_joined

,user_id,name,email,signup_date,is_dupe,event_id,user_email,event_type,timestamp,repr_print,cleaned_email,encoded_email_repr
158,<NA>,NaN,NaN,NaN,NaN,159,amy.robinson@outlook.com,deleted_account,2024-04-24 23:18:50.063910,' amy.robinson@outlook.com ',amy.robinson@outlook.com,'amy.robinson@outlook.com'
262,<NA>,NaN,NaN,NaN,NaN,263,andrea.calderon@outlook.com,logout,2023-08-02 22:25:58.806733,' andrea.calderon@outlook.com ',andrea.calderon@outlook.com,'andrea.calderon@outlook.com'
105,<NA>,NaN,NaN,NaN,NaN,106,barbara.scott@outlook.com,updated_profile,2024-02-14 01:33:05.969064,' barbara.scott@outlook.com ',barbara.scott@outlook.com,'barbara.scott@outlook.com'
20,<NA>,NaN,NaN,NaN,NaN,21,chelsea.jackson@live.com,password_reset,2023-07-10 10:40:06.963536,' chelsea.jackson@live.com ',chelsea.jackson@live.com,'chelsea.jackson@live.com'
193,<NA>,NaN,NaN,NaN,NaN,194,chelsea.jackson@live.com,password_reset,2023-05-31 09:44:45.738924,' chelsea.jackson@live.com ',chelsea.jackson@live.com,'chelsea.jackson@live.com'
...,...,...,...,...,...,...,...,...,...,...,...,...
98,<NA>,NaN,NaN,NaN,NaN,99,NaN,purchase,2023-06-16 05:11:23.034739,nan,nan,'nan'
280,<NA>,NaN,NaN,NaN,NaN,281,NaN,purchase,2025-02-10 15:34:22.668397,nan,nan,'nan'
357,<NA>,NaN,NaN,NaN,NaN,358,NaN,password_reset,2024-01-08 01:21:25.623984,nan,nan,'nan'
396,<NA>,NaN,NaN,NaN,NaN,397,NaN,password_reset,2024-08-22 04:55:50.082282,nan,nan,'nan'


In [411]:
# I will apply the above to the email columns of the original dataframes
# We can then join and retry the join and check the new join rate
# I noticed that the remaining unmatched emails are missing dots in between the first and last names as well as missing dots between the domain names and .com
events_df['user_email'] =  events_df['user_email'].astype(str).map(lambda x: re.sub(r'[^\x00-\x7F]+', '', x)).map(lambda x: x.strip()).map(lambda x: x.lower())
users_df['email'] =  users_df['email'].astype(str).map(lambda x: re.sub(r'[^\x00-\x7F]+', '', x)).map(lambda x: x.strip()).map(lambda x: x.lower())

joined_df_cleaned = users_df.merge(events_df, how="right", left_on='email', right_on='user_email')

nanrows_joined_cleaned = joined_df_cleaned[ joined_df_cleaned[['email', 'user_email']].isnull().any(axis=1) ].sort_values(by=['user_email'])
nanrows_joined_cleaned


,user_id,name,email,signup_date,is_dupe,event_id,user_email,event_type,timestamp
337,NaN,NaN,NaN,NaN,NaN,338,andreapatterson@icloudcom,login,2023-11-26 13:22:27.872263
498,NaN,NaN,NaN,NaN,NaN,499,barbarascott@outlookcom,login,2024-12-18 17:08:01.895173
84,NaN,NaN,NaN,NaN,NaN,85,chadbeck@comcastnet,purchase,2023-12-13 20:28:49.048561
223,NaN,NaN,NaN,NaN,NaN,224,davidparker@outlookcom,password_reset,2023-08-19 19:36:13.211571
461,NaN,NaN,NaN,NaN,NaN,462,jadecarr@gmailcom,login,2023-05-25 10:06:49.957187
1,NaN,NaN,NaN,NaN,NaN,2,jenniferevans@msncom,purchase,2023-12-03 23:55:36.554694
310,NaN,NaN,NaN,NaN,NaN,311,jessicanunez@gmailcom,password_reset,2024-11-21 23:03:16.052579
307,NaN,NaN,NaN,NaN,NaN,308,johndouglas@livecom,updated_profile,2023-05-11 03:01:35.589453
290,NaN,NaN,NaN,NaN,NaN,291,josephbowers@comcastnet,password_reset,2023-11-01 20:25:13.251929
361,NaN,NaN,NaN,NaN,NaN,362,lesliewalton@yahoocom,logout,2024-11-08 03:53:38.624240


In [412]:
# I used barbarascott@outlookcom as an example by looking up the first name
# It looks like it is available as a user, and I will assume these are just mistakes in formatting in the events

users_df[ users_df['email'].str.contains('barbara', na=False)  ]

,user_id,name,email,signup_date,is_dupe
61,61,Barbara Scott,barbara.scott@outlook.com,2025-02-12,False


In [413]:
# I will extract first+last names from the users table and use those to add the dots
# define a set of pre/postfixes to exclude
# im assuming the emails dont have prefixes like Mr or postfixes like Jr

# Split on space
# users_df[ users_df['name'].str.contains('Mr', na=False)  ]
users_df[['first_name', 'last_name']] = users_df['name'].str.split(' ', n=1, expand=True)
users_df['first_name']=users_df['first_name'].map(lambda x: x.strip())
users_df['last_name']=users_df['last_name'].map(lambda x: x.strip())
users_df

,user_id,name,email,signup_date,is_dupe,first_name,last_name
0,0,Danielle Johnson,danielle.johnson@gmail.com,2024-09-27,False,Danielle,Johnson
1,1,Jeffery Wagner,jeffery.wagner@icloud.com,2024-07-26,False,Jeffery,Wagner
2,2,Amy Robinson,amy.robinson@outlook.com,2023-04-12,False,Amy,Robinson
3,3,Linda Wolfe,linda.wolfe@gmail.com,2025-03-07,False,Linda,Wolfe
4,4,Matthew Davis,matthew.davis@hotmail.com,2024-06-22,False,Matthew,Davis
...,...,...,...,...,...,...,...
97,97,Gregory Estes White,gregory.estes@gmail.com,2024-02-19,False,Gregory,Estes White
98,98,Brent Wright White,brent.wright@msn.com,2024-11-13,False,Brent,Wright White
99,99,Ms. Dana Cannon,dana.cannon@outlook.com,2023-10-25,False,Ms.,Dana Cannon
100,100,Rick Daniels-Brown,rick.daniels@outlook.com,2024-02-09,False,Rick,Daniels-Brown


In [414]:
# It looks like some names have some interesting properties
# For prefixes, we have Mrs. and Ms.
# Some people have middle names, Like Michelle Pierce White
# It looks like the emails take firstname.middlename like anna.baker@hotmail.com even though her full name is Anna Baker Lee
users_df[ users_df['last_name'].str.contains(' ', na=False)  ]


,user_id,name,email,signup_date,is_dupe,first_name,last_name
87,87,Mrs. Erica Mcclain,erica.mcclain@icloud.com,2023-04-29,False,Mrs.,Erica Mcclain
88,88,Andrea Calderon Lee,andrea.calderon@outlook.com,2024-06-03,False,Andrea,Calderon Lee
89,89,Anna Baker Lee,anna.baker@hotmail.com,2024-01-24,False,Anna,Baker Lee
91,91,Jason Ryan Brown,jason.ryan@comcast.net,2023-11-21,False,Jason,Ryan Brown
93,93,Michelle Pierce White,michelle.pierce@icloud.com,2024-12-09,False,Michelle,Pierce White
94,94,Wendy Peters White,wendy.peters@msn.com,2023-11-10,False,Wendy,Peters White
96,96,Laura Perez White,laura.perez@live.com,2023-12-03,False,Laura,Perez White
97,97,Gregory Estes White,gregory.estes@gmail.com,2024-02-19,False,Gregory,Estes White
98,98,Brent Wright White,brent.wright@msn.com,2024-11-13,False,Brent,Wright White
99,99,Ms. Dana Cannon,dana.cannon@outlook.com,2023-10-25,False,Ms.,Dana Cannon


In [415]:
# also checking for hyphens in the last name
users_df[ users_df['last_name'].str.contains('-', na=False)  ]

,user_id,name,email,signup_date,is_dupe,first_name,last_name
90,90,Mario Smith-Brown,mario.smith@aol.com,2024-12-05,False,Mario,Smith-Brown
92,92,James Davis-Smith,james.davis@outlook.com,2024-03-14,False,James,Davis-Smith
95,95,Lisa Evans-Lee,lisa.evans@msn.com,2023-07-15,False,Lisa,Evans-Lee
100,100,Rick Daniels-Brown,rick.daniels@outlook.com,2024-02-09,False,Rick,Daniels-Brown


In [416]:
# We will remove the prefixes in a more exact manner by specifying them in a list and removing manually
# The names will be split into 3 based on ' ' and the first two will be assumed to be the email

remove_prefix_postfix = '|'.join(['Mrs.', 'Ms.', 'Mr.', 'Jr.', 'Dr.'])
users_df['name'] = users_df['name'].str.strip().replace(remove_prefix_postfix, '', regex=True).str.strip()
users_df

,user_id,name,email,signup_date,is_dupe,first_name,last_name
0,0,Danielle Johnson,danielle.johnson@gmail.com,2024-09-27,False,Danielle,Johnson
1,1,Jeffery Wagner,jeffery.wagner@icloud.com,2024-07-26,False,Jeffery,Wagner
2,2,Amy Robinson,amy.robinson@outlook.com,2023-04-12,False,Amy,Robinson
3,3,Linda Wolfe,linda.wolfe@gmail.com,2025-03-07,False,Linda,Wolfe
4,4,Matthew Davis,matthew.davis@hotmail.com,2024-06-22,False,Matthew,Davis
...,...,...,...,...,...,...,...
97,97,Gregory Estes White,gregory.estes@gmail.com,2024-02-19,False,Gregory,Estes White
98,98,Brent Wright White,brent.wright@msn.com,2024-11-13,False,Brent,Wright White
99,99,Dana Cannon,dana.cannon@outlook.com,2023-10-25,False,Ms.,Dana Cannon
100,100,Rick Daniels-Brown,rick.daniels@outlook.com,2024-02-09,False,Rick,Daniels-Brown


In [417]:
users_df[['first_name', 'last_name', 'extra_last_name']] = users_df['name'].str.split(' ', n=3, expand=True)
users_df['first_name']=users_df['first_name'].map(lambda x: x.strip())
users_df['last_name']=users_df['last_name'].map(lambda x: x.strip())
users_df

,user_id,name,email,signup_date,is_dupe,first_name,last_name,extra_last_name
0,0,Danielle Johnson,danielle.johnson@gmail.com,2024-09-27,False,Danielle,Johnson,None
1,1,Jeffery Wagner,jeffery.wagner@icloud.com,2024-07-26,False,Jeffery,Wagner,None
2,2,Amy Robinson,amy.robinson@outlook.com,2023-04-12,False,Amy,Robinson,None
3,3,Linda Wolfe,linda.wolfe@gmail.com,2025-03-07,False,Linda,Wolfe,None
4,4,Matthew Davis,matthew.davis@hotmail.com,2024-06-22,False,Matthew,Davis,None
...,...,...,...,...,...,...,...,...
97,97,Gregory Estes White,gregory.estes@gmail.com,2024-02-19,False,Gregory,Estes,White
98,98,Brent Wright White,brent.wright@msn.com,2024-11-13,False,Brent,Wright,White
99,99,Dana Cannon,dana.cannon@outlook.com,2023-10-25,False,Dana,Cannon,None
100,100,Rick Daniels-Brown,rick.daniels@outlook.com,2024-02-09,False,Rick,Daniels-Brown,None


In [418]:
# Some last names are hyphenated, so I will split those out by '-' and treat the first one as the last name
# This seems to be how the emails are formatted
users_df[['last_name', 'extra_last_name']] = users_df['last_name'].str.split('-', n=3, expand=True)
users_df['first_name']=users_df['first_name'].map(lambda x: x.strip())
users_df['last_name']=users_df['last_name'].map(lambda x: x.strip())
users_df.sort_values('name')

,user_id,name,email,signup_date,is_dupe,first_name,last_name,extra_last_name
48,48,Adam Wise,adam.wise@live.com,2024-09-30,False,Adam,Wise,None
16,16,Alexander Le,alexander.le@icloud.com,2023-10-20,False,Alexander,Le,None
62,62,Amanda Ball,amanda.ball@yahoo.com,2023-12-30,False,Amanda,Ball,None
2,2,Amy Robinson,amy.robinson@outlook.com,2023-04-12,False,Amy,Robinson,None
88,88,Andrea Calderon Lee,andrea.calderon@outlook.com,2024-06-03,False,Andrea,Calderon,None
...,...,...,...,...,...,...,...,...
22,22,Todd Jones,todd.jones@outlook.com,2023-07-10,False,Todd,Jones,None
9,9,Veronica Bowman,veronica.bowman@msn.com,2024-12-14,False,Veronica,Bowman,None
24,24,Wanda Santos,wanda.santos@hotmail.com,2023-07-05,False,Wanda,Santos,None
42,42,Wendy Jones,wendy.jones@hotmail.com,2025-02-18,False,Wendy,Jones,None


In [419]:
# concatenating the first and last names in the user table for easier comparison later
users_df['names_concat'] = users_df['first_name'].str.lower() + users_df['last_name'].str.lower()
users_df['names_concat_dot'] = users_df['first_name'].str.lower() + '.' + users_df['last_name'].str.lower()
users_df

,user_id,name,email,signup_date,is_dupe,first_name,last_name,extra_last_name,names_concat,names_concat_dot
0,0,Danielle Johnson,danielle.johnson@gmail.com,2024-09-27,False,Danielle,Johnson,None,daniellejohnson,danielle.johnson
1,1,Jeffery Wagner,jeffery.wagner@icloud.com,2024-07-26,False,Jeffery,Wagner,None,jefferywagner,jeffery.wagner
2,2,Amy Robinson,amy.robinson@outlook.com,2023-04-12,False,Amy,Robinson,None,amyrobinson,amy.robinson
3,3,Linda Wolfe,linda.wolfe@gmail.com,2025-03-07,False,Linda,Wolfe,None,lindawolfe,linda.wolfe
4,4,Matthew Davis,matthew.davis@hotmail.com,2024-06-22,False,Matthew,Davis,None,matthewdavis,matthew.davis
...,...,...,...,...,...,...,...,...,...,...
97,97,Gregory Estes White,gregory.estes@gmail.com,2024-02-19,False,Gregory,Estes,None,gregoryestes,gregory.estes
98,98,Brent Wright White,brent.wright@msn.com,2024-11-13,False,Brent,Wright,None,brentwright,brent.wright
99,99,Dana Cannon,dana.cannon@outlook.com,2023-10-25,False,Dana,Cannon,None,danacannon,dana.cannon
100,100,Rick Daniels-Brown,rick.daniels@outlook.com,2024-02-09,False,Rick,Daniels,Brown,rickdaniels,rick.daniels


In [420]:
# extracting the first/lastname portions of the email for easy comparison later
events_df['names_extract'] = events_df['user_email'].apply(lambda x: x.split('@')[0])
events_df

,event_id,user_email,event_type,timestamp,names_extract
0,1,daniel.burton@protonmail.com,logout,2023-09-10 20:32:58.421093,daniel.burton
1,2,jenniferevans@msncom,purchase,2023-12-03 23:55:36.554694,jenniferevans
2,3,jessica.nunez@gmail.com,logout,2024-10-16 07:20:12.854738,jessica.nunez
3,4,tanya.house@protonmail.com,password_reset,2024-01-11 04:28:54.790473,tanya.house
4,5,lisa.evans@msn.com,login,2024-10-09 13:41:53.942650,lisa.evans
...,...,...,...,...,...
495,496,justin.jordan@outlook.com,updated_profile,2024-01-25 01:48:58.194437,justin.jordan
496,497,devon.hall@comcast.net,logout,2023-08-25 21:27:20.373314,devon.hall
497,498,tanya.house@protonmail.com,viewed_product,2023-07-07 11:28:07.010142,tanya.house
498,499,barbarascott@outlookcom,login,2024-12-18 17:08:01.895173,barbarascott


In [421]:
# now we have extracted the first and last names we need to fix some of the missing emails from before
# In the joined dataframe, we can look at events that havent matched (null user info)
# for each event, check the users table to see if the person's name exists
# if the name exists then replace the part of the email before the @ with first_name.last_name
# replace com or net with .com or .net

joined_df_cleaned_names_separated = users_df.merge(events_df, how="right", left_on='email', right_on='user_email')

nanrows_joined_cleaned_names_separated = joined_df_cleaned_names_separated[ joined_df_cleaned_names_separated[['email']].isnull().any(axis=1) ].sort_values(by=['user_email'])
nanrows_joined_cleaned_names_separated

,user_id,name,email,signup_date,is_dupe,first_name,last_name,extra_last_name,names_concat,names_concat_dot,event_id,user_email,event_type,timestamp,names_extract
337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,338,andreapatterson@icloudcom,login,2023-11-26 13:22:27.872263,andreapatterson
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,499,barbarascott@outlookcom,login,2024-12-18 17:08:01.895173,barbarascott
84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,chadbeck@comcastnet,purchase,2023-12-13 20:28:49.048561,chadbeck
223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,davidparker@outlookcom,password_reset,2023-08-19 19:36:13.211571,davidparker
461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,462,jadecarr@gmailcom,login,2023-05-25 10:06:49.957187,jadecarr
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,jenniferevans@msncom,purchase,2023-12-03 23:55:36.554694,jenniferevans
310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311,jessicanunez@gmailcom,password_reset,2024-11-21 23:03:16.052579,jessicanunez
307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308,johndouglas@livecom,updated_profile,2023-05-11 03:01:35.589453,johndouglas
290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,291,josephbowers@comcastnet,password_reset,2023-11-01 20:25:13.251929,josephbowers
361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,362,lesliewalton@yahoocom,logout,2024-11-08 03:53:38.624240,lesliewalton


In [422]:
# We want to check only the events where email in the user table is null
# where its null, mark the new column matched_name as true
# we will use this column later to single out those incorrect emails and fix them, the assumption being that some formatting mistake happened
# they also happen to be the ones with incorrect domains so we can fix those as well

mask = joined_df_cleaned_names_separated['email'].isnull()
joined_df_cleaned_names_separated.loc[mask, 'matched_name'] = joined_df_cleaned_names_separated.loc[mask, 'names_extract'].apply(
    lambda x: any(name in str(x).lower() for name in users_df['names_concat'].str.lower())
)
joined_df_cleaned_names_separated[ joined_df_cleaned_names_separated['matched_name'] == True]

,user_id,name,email,signup_date,is_dupe,first_name,last_name,extra_last_name,names_concat,names_concat_dot,event_id,user_email,event_type,timestamp,names_extract,matched_name
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,jenniferevans@msncom,purchase,2023-12-03 23:55:36.554694,jenniferevans,True
84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,chadbeck@comcastnet,purchase,2023-12-13 20:28:49.048561,chadbeck,True
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102,nancybaker@icloudcom,viewed_product,2023-11-18 22:04:48.708847,nancybaker,True
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108,michelewalker@msncom,logout,2023-12-06 21:05:50.273419,michelewalker,True
223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,davidparker@outlookcom,password_reset,2023-08-19 19:36:13.211571,davidparker,True
285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,286,phyllismanning@msncom,purchase,2024-11-06 07:49:56.047680,phyllismanning,True
290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,291,josephbowers@comcastnet,password_reset,2023-11-01 20:25:13.251929,josephbowers,True
301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,302,michellecherry@icloudcom,updated_profile,2023-11-30 07:04:08.783915,michellecherry,True
307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308,johndouglas@livecom,updated_profile,2023-05-11 03:01:35.589453,johndouglas,True
310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311,jessicanunez@gmailcom,password_reset,2024-11-21 23:03:16.052579,jessicanunez,True


In [423]:
emails_to_fix = joined_df_cleaned_names_separated[ joined_df_cleaned_names_separated['matched_name'] == True]
emails_to_fix.sort_values(by='user_email')

,user_id,name,email,signup_date,is_dupe,first_name,last_name,extra_last_name,names_concat,names_concat_dot,event_id,user_email,event_type,timestamp,names_extract,matched_name
337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,338,andreapatterson@icloudcom,login,2023-11-26 13:22:27.872263,andreapatterson,True
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,499,barbarascott@outlookcom,login,2024-12-18 17:08:01.895173,barbarascott,True
84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,chadbeck@comcastnet,purchase,2023-12-13 20:28:49.048561,chadbeck,True
223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,davidparker@outlookcom,password_reset,2023-08-19 19:36:13.211571,davidparker,True
461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,462,jadecarr@gmailcom,login,2023-05-25 10:06:49.957187,jadecarr,True
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,jenniferevans@msncom,purchase,2023-12-03 23:55:36.554694,jenniferevans,True
310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311,jessicanunez@gmailcom,password_reset,2024-11-21 23:03:16.052579,jessicanunez,True
307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308,johndouglas@livecom,updated_profile,2023-05-11 03:01:35.589453,johndouglas,True
290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,291,josephbowers@comcastnet,password_reset,2023-11-01 20:25:13.251929,josephbowers,True
361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,362,lesliewalton@yahoocom,logout,2024-11-08 03:53:38.624240,lesliewalton,True


In [424]:
# use the above list to identify emails in the events table that are not formed properly
# use that list to derive a lookup table from the users table using the concatenated names
# fix them in-place in the events table as described above using this lookup table

users_with_broken_emails_in_events = users_df[ users_df['names_concat'].isin(emails_to_fix['names_extract']) ]
users_with_broken_emails_in_events.sort_values(by='email')

,user_id,name,email,signup_date,is_dupe,first_name,last_name,extra_last_name,names_concat,names_concat_dot
49,49,Andrea Patterson,andrea.patterson@icloud.com,2025-01-12,False,Andrea,Patterson,None,andreapatterson,andrea.patterson
61,61,Barbara Scott,barbara.scott@outlook.com,2025-02-12,False,Barbara,Scott,None,barbarascott,barbara.scott
63,63,Chad Beck,chad.beck@comcast.net,2024-12-17,False,Chad,Beck,None,chadbeck,chad.beck
51,51,David Parker,david.parker@outlook.com,2024-12-27,False,David,Parker,None,davidparker,david.parker
44,44,Jade Carr,jade.carr@gmail.com,2023-04-13,False,Jade,Carr,None,jadecarr,jade.carr
64,64,Jennifer Evans,jennifer.evans@msn.com,2024-01-27,False,Jennifer,Evans,None,jenniferevans,jennifer.evans
29,29,Jessica Nunez,jessica.nunez@gmail.com,2024-01-27,False,Jessica,Nunez,None,jessicanunez,jessica.nunez
73,73,John Douglas,john.douglas@live.com,2024-10-21,False,John,Douglas,None,johndouglas,john.douglas
34,34,Joseph Bowers,joseph.bowers@comcast.net,2025-03-07,False,Joseph,Bowers,None,josephbowers,joseph.bowers
41,41,Leslie Walton,leslie.walton@yahoo.com,2025-04-06,False,Leslie,Walton,None,lesliewalton,leslie.walton


In [425]:
#First clean up the domains
# if there are more domains we can make a more complex function later to handle that replacement

mask = events_df['names_extract'].isin(users_with_broken_emails_in_events['names_concat'])

events_df.loc[mask, 'user_email'] = events_df.loc[mask, 'user_email'].str.replace(r'com$', '.com', regex=True)
events_df.loc[mask, 'user_email'] = events_df.loc[mask, 'user_email'].str.replace(r'net$', '.net', regex=True)
events_df[mask]



,event_id,user_email,event_type,timestamp,names_extract
1,2,jenniferevans@msn.com,purchase,2023-12-03 23:55:36.554694,jenniferevans
84,85,chadbeck@comcast.net,purchase,2023-12-13 20:28:49.048561,chadbeck
101,102,nancybaker@icloud.com,viewed_product,2023-11-18 22:04:48.708847,nancybaker
107,108,michelewalker@msn.com,logout,2023-12-06 21:05:50.273419,michelewalker
223,224,davidparker@outlook.com,password_reset,2023-08-19 19:36:13.211571,davidparker
285,286,phyllismanning@msn.com,purchase,2024-11-06 07:49:56.047680,phyllismanning
290,291,josephbowers@comcast.net,password_reset,2023-11-01 20:25:13.251929,josephbowers
301,302,michellecherry@icloud.com,updated_profile,2023-11-30 07:04:08.783915,michellecherry
307,308,johndouglas@live.com,updated_profile,2023-05-11 03:01:35.589453,johndouglas
310,311,jessicanunez@gmail.com,password_reset,2024-11-21 23:03:16.052579,jessicanunez


In [426]:
# split out the email to before the @ and after the @ to isolate the now fixed domains
# make a new column with the names_concat_dot information from users
# replace the user_email for the malformed emails with a concatenation between the two

events_df.loc[mask, 'user_email_split'] = events_df.loc[mask,'user_email'].apply(lambda x: x.split('@')[1])

events_df = events_df.merge(
    users_df[['names_concat', 'names_concat_dot']],
    left_on='names_extract',
    right_on='names_concat',
    how='left'
)

events_df.drop(columns=['names_concat'], inplace=True)

events_df.loc[mask, 'user_email'] = events_df.loc[mask,'names_concat_dot'] + '@' + events_df.loc[mask,'user_email_split']

events_df[mask]

,event_id,user_email,event_type,timestamp,names_extract,user_email_split,names_concat_dot
1,2,jennifer.evans@msn.com,purchase,2023-12-03 23:55:36.554694,jenniferevans,msn.com,jennifer.evans
84,85,chad.beck@comcast.net,purchase,2023-12-13 20:28:49.048561,chadbeck,comcast.net,chad.beck
101,102,nancy.baker@icloud.com,viewed_product,2023-11-18 22:04:48.708847,nancybaker,icloud.com,nancy.baker
107,108,michele.walker@msn.com,logout,2023-12-06 21:05:50.273419,michelewalker,msn.com,michele.walker
223,224,david.parker@outlook.com,password_reset,2023-08-19 19:36:13.211571,davidparker,outlook.com,david.parker
285,286,phyllis.manning@msn.com,purchase,2024-11-06 07:49:56.047680,phyllismanning,msn.com,phyllis.manning
290,291,joseph.bowers@comcast.net,password_reset,2023-11-01 20:25:13.251929,josephbowers,comcast.net,joseph.bowers
301,302,michelle.cherry@icloud.com,updated_profile,2023-11-30 07:04:08.783915,michellecherry,icloud.com,michelle.cherry
307,308,john.douglas@live.com,updated_profile,2023-05-11 03:01:35.589453,johndouglas,live.com,john.douglas
310,311,jessica.nunez@gmail.com,password_reset,2024-11-21 23:03:16.052579,jessicanunez,gmail.com,jessica.nunez


In [427]:
events_df

,event_id,user_email,event_type,timestamp,names_extract,user_email_split,names_concat_dot
0,1,daniel.burton@protonmail.com,logout,2023-09-10 20:32:58.421093,daniel.burton,NaN,NaN
1,2,jennifer.evans@msn.com,purchase,2023-12-03 23:55:36.554694,jenniferevans,msn.com,jennifer.evans
2,3,jessica.nunez@gmail.com,logout,2024-10-16 07:20:12.854738,jessica.nunez,NaN,NaN
3,4,tanya.house@protonmail.com,password_reset,2024-01-11 04:28:54.790473,tanya.house,NaN,NaN
4,5,lisa.evans@msn.com,login,2024-10-09 13:41:53.942650,lisa.evans,NaN,NaN
...,...,...,...,...,...,...,...
495,496,justin.jordan@outlook.com,updated_profile,2024-01-25 01:48:58.194437,justin.jordan,NaN,NaN
496,497,devon.hall@comcast.net,logout,2023-08-25 21:27:20.373314,devon.hall,NaN,NaN
497,498,tanya.house@protonmail.com,viewed_product,2023-07-07 11:28:07.010142,tanya.house,NaN,NaN
498,499,barbara.scott@outlook.com,login,2024-12-18 17:08:01.895173,barbarascott,outlook.com,barbara.scott


In [428]:
print(users_df.head())

   user_id              name                       email signup_date  is_dupe  \
0        0  Danielle Johnson  danielle.johnson@gmail.com  2024-09-27    False   
1        1    Jeffery Wagner   jeffery.wagner@icloud.com  2024-07-26    False   
2        2      Amy Robinson    amy.robinson@outlook.com  2023-04-12    False   
3        3       Linda Wolfe       linda.wolfe@gmail.com  2025-03-07    False   
4        4     Matthew Davis   matthew.davis@hotmail.com  2024-06-22    False   

  first_name last_name extra_last_name     names_concat  names_concat_dot  
0   Danielle   Johnson            None  daniellejohnson  danielle.johnson  
1    Jeffery    Wagner            None    jefferywagner    jeffery.wagner  
2        Amy  Robinson            None      amyrobinson      amy.robinson  
3      Linda     Wolfe            None       lindawolfe       linda.wolfe  
4    Matthew     Davis            None     matthewdavis     matthew.davis  


In [429]:
# Now we should have a very clean event data set
# We can join once more and check out the merge rate
# The onl events that did not match are the five with no user emails in the events table
# Our cleanup efforts paid off
joined_df_cleaned_final = users_df.merge(events_df, how="right", left_on='email', right_on='user_email')

nanrows_joined_cleaned_final = joined_df_cleaned_final[ joined_df_cleaned_final[['email']].isnull().any(axis=1) ].sort_values(by=['user_email'])
nanrows_joined_cleaned_final['event_id']


98      99
280    281
357    358
396    397
420    421
Name: event_id, dtype: int64

In [ ]:
joined_df_cleaned_final['user_id'] = joined_df_cleaned_final['user_id'].astype('Int64')  # capital "I" for pandas nullable integer

joined_df_cleaned_final = joined_df_cleaned_final[['user_id', 'first_name', 'last_name', 'event_type', 'timestamp', 'signup_date']]

In [ ]:
joined_df_cleaned_final.to_csv('joined_events.csv', index=False)